## Connect with Google Drive

In [1]:
!pip install gdown

In [10]:
! rm -rf video.mp4

In [11]:
import gdown

url = "https://drive.google.com/uc?id=17Hmj4w9DXN-jGF1NfzIs5jJ7hMYPwkJG"

output = 'video_1.mp4'

gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=17Hmj4w9DXN-jGF1NfzIs5jJ7hMYPwkJG
To: /kaggle/working/video_1.mp4
100%|██████████| 16.8M/16.8M [00:00<00:00, 39.8MB/s]


'video_1.mp4'

In [12]:
import gdown

url = "https://drive.google.com/uc?id=1ik16ZuPR_nqaYV6OxIPSU-dPpFmNpySp"

output = 'video_2.mp4'

gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1ik16ZuPR_nqaYV6OxIPSU-dPpFmNpySp
To: /kaggle/working/video_2.mp4
100%|██████████| 101M/101M [00:00<00:00, 154MB/s] 


'video_2.mp4'

In [13]:
SOURCE_VIDEO_PATH_1 = "/kaggle/working/video_1.mp4"
SOURCE_VIDEO_PATH_2 = "/kaggle/working/video_2.mp4"

## Check GPU access

In [5]:
!nvidia-smi

Sat Feb 17 14:06:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


## Install YOLOv8

In [7]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5432.1/8062.4 GB disk)


## Install Roboflow Supervision

In [8]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.3.0


## Load pre-trained YOLOv8 model

In [14]:
%cd {HOME}

/kaggle/working


In [15]:
import gdown

url = "https://drive.google.com/uc?id=11CVHlga1WsmH3MsytW-ItWiCGwYZ3G98"

output = '/kaggle/working/trained_model.pt'

gdown.download(url, output)

Downloading...
From (original): https://drive.google.com/uc?id=11CVHlga1WsmH3MsytW-ItWiCGwYZ3G98
From (redirected): https://drive.google.com/uc?id=11CVHlga1WsmH3MsytW-ItWiCGwYZ3G98&confirm=t&uuid=80f444bc-baa2-493d-b9fe-8a081a3bbc6e
To: /kaggle/working/trained_model.pt
100%|██████████| 87.7M/87.7M [00:00<00:00, 184MB/s] 


'/kaggle/working/trained_model.pt'

In [16]:
from ultralytics import YOLO

model = YOLO(os.path.relpath("/kaggle/working/trained_model.pt"))
model.fuse()

Model summary (fused): 268 layers, 43609692 parameters, 0 gradients, 164.8 GFLOPs


## Track and Count

In [17]:
import supervision as sv

#LINE_START = sv.Point(0, 900)
#LINE_END = sv.Point(1000, 480)

In [18]:
TARGET_VIDEO_PATH = f"/kaggle/working/output_video.mp4"

In [20]:
import numpy as np

# initiate polygon zone
polygon = np.array([
    [50,1000],
    [1050, 1000],
    [1050, 300],
    [50, 300]
])

In [51]:
list_of_dict = []
in_count = 0
out_count = 0
in_check = False
out_check = False

In [52]:
import matplotlib.pyplot as plt
from PIL import Image

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH_2)

# create frame generator
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH_2)

# create LineCounter instance
# line_counter = sv.LineZone(start=LINE_START, end=LINE_END)

# create ZoneCounter instance
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# create ZoneCounter instance
zone_2 = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# create instance of BoxAnnotator, LineCounterAnnotator and ZoneCounterAnnotator
# line_annotator = sv.LineZoneAnnotator(
#    thickness=4, 
#    text_thickness=4, 
#    text_scale=2
#)

box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

zone_annotator = sv.PolygonZoneAnnotator(
    zone=zone, 
    color=sv.Color.white(), 
    thickness=6, 
    text_thickness=6, 
    text_scale=2
)

zone_annotator_2 = sv.PolygonZoneAnnotator(
    zone=zone_2, 
    color=sv.Color.blue(), 
    thickness=6, 
    text_thickness=6, 
    text_scale=2
)

# open target video file
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:  
    for result in model.track(source=SOURCE_VIDEO_PATH_2, tracker = 'bytetrack.yaml', show=False, stream=True, agnostic_nms=True, persist=True ):

        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)
        
        labels = [
            f"{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]

        frame = box_annotator.annotate(
            scene=frame, 
            detections=detections,
            labels=labels
        )
        
        detections = detections[detections.class_id == 2]
        zone.trigger(detections=detections)
        
        if (any(detections.class_id == 2) and  any(zone.trigger(detections=detections)) and in_check == False):
            in_count = in_count + 1
            print(in_count)
            in_check = True
            out_check = False
            #dict_entry = {"class_id": detections.class_id, "track_id": detections.tracker_id}
            #list_of_dict.append(dict_entry)
        
        detections = sv.Detections.from_yolov8(result)
        detections = detections[detections.class_id == 3]
        zone_2.trigger(detections=detections)
        
        if (any(detections.class_id == 3) and  any(zone_2.trigger(detections=detections)) and out_check == False):
            out_count = out_count + 1
            print(out_count)
            out_check = True
            in_check = False
            #dict_entry = {"class_id": detections.class_id, "track_id": detections.tracker_id}
            #list_of_dict.append(dict_entry)
        
        #line_counter.trigger(detections=detections)
        #line_annotator.annotate(frame=frame, line_counter=line_counter)
        frame = zone_annotator.annotate(scene=frame)
        frame = zone_annotator_2.annotate(scene=frame)
        
        #plt.imshow(frame)
        #plt.show()
        #print(detections.class_id)
        #print(detections.tracker_id)
        
        sink.write_frame(frame)


video 1/1 (1/1325) /kaggle/working/video_2.mp4: 480x800 3 sockets, 66.4ms
video 1/1 (2/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 65.4ms
video 1/1 (3/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 50.4ms
video 1/1 (4/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 41.7ms
video 1/1 (5/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 40.6ms
video 1/1 (6/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 32.9ms
video 1/1 (7/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.0ms
video 1/1 (8/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.2ms
video 1/1 (9/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.0ms
video 1/1 (10/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.8ms
video 1/1 (11/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.8ms
video 1/1 (12/1325) /kaggle/working/video_2.mp4: 480x800 1 main

1
True
False


video 1/1 (31/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.0ms
video 1/1 (32/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.4ms
video 1/1 (33/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.4ms
video 1/1 (34/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 31.8ms
video 1/1 (35/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.5ms
video 1/1 (36/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 32.4ms
video 1/1 (37/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.3ms
video 1/1 (38/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.5ms
video 1/1 (39/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.2ms
video 1/1 (40/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 33.9ms
video 1/1 (41/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 3 sockets, 34.0ms
video 1/1 (42/1325) /kaggle/working/video_2

1
False
True


video 1/1 (48/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 32.2ms
video 1/1 (49/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 33.4ms
video 1/1 (50/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 32.9ms
video 1/1 (51/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 33.0ms
video 1/1 (52/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 32.7ms
video 1/1 (53/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 31.8ms
video 1/1 (54/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 32.8ms
video 1/1 (55/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 33.0ms
video 1/1 (56/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 2 sockets, 1 socket_with_box, 33.3ms
video 1/1 (57/1325) /kaggle/working/video_2.mp

...

video 1/1 (969/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 32.9ms
video 1/1 (970/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 32.7ms
video 1/1 (971/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 33.0ms
video 1/1 (972/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 33.2ms
video 1/1 (973/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 33.2ms
video 1/1 (974/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 32.7ms
video 1/1 (975/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 33.3ms
video 1/1 (976/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 33.1ms
video 1/1 (977/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 1 socket_with_box, 33.0ms
video 1/1 (978/1325) /kaggle/working/video_2.m

7
True
False


video 1/1 (1161/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 33.3ms
video 1/1 (1162/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 32.9ms
video 1/1 (1163/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 33.5ms
video 1/1 (1164/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 33.7ms
video 1/1 (1165/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 33.6ms
video 1/1 (1166/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 33.3ms
video 1/1 (1167/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 32.5ms
video 1/1 (1168/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 32.7ms
video 1/1 (1169/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 32.8ms
video 1/1 (1170/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 32.5ms
video 1/1 (1171/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket, 33.4ms
video 1/1 (1172/1325) /kaggle/wo

7
False
True


video 1/1 (1187/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 32.5ms
video 1/1 (1188/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 32.3ms
video 1/1 (1189/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 33.6ms
video 1/1 (1190/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 33.4ms
video 1/1 (1191/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 33.3ms
video 1/1 (1192/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 33.0ms
video 1/1 (1193/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 33.0ms
video 1/1 (1194/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 32.9ms
video 1/1 (1195/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 32.9ms
video 1/1 (1196/1325) /kaggle/working/video_2.mp4: 480x800 1 main_box, 1 socket_with_box, 33.4ms
video 1/1 (1197/1325) /kaggle/

In [53]:
print(in_count)
print(out_count)

7
7


In [38]:
for dictionary in list_of_dict:
    print(dictionary)

In [32]:
from IPython.display import FileLink
FileLink(r'output_video.mp4')

/kaggle/working/output_video.mp4